In [0]:
import pandas as pd
import requests
import json


df = pd.read_csv('https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv')

In [0]:
df.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


In [0]:
cid = ['B19001', 'B01001', 'B03002']
cidstring = ",".join(cid)

In [0]:
geoid = '31000US' + df['CBSA'].astype(str)
df['geoid'] = geoid

In [0]:
df.head()

,CBSA,NAME,POPESTIMATE2017,geoid
0,10180,"Abilene, TX",170219,31000US10180
1,10420,"Akron, OH",703505,31000US10420
2,10500,"Albany, GA",151434,31000US10500
3,10540,"Albany, OR",125047,31000US10540
4,10580,"Albany-Schenectady-Troy, NY",886188,31000US10580


In [0]:
geoidstring_part1 = ",".join(geoid[:int(len(geoid)/2)])
requesturl1 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (cidstring, geoidstring_part1)
loadedjson1 = requests.get(requesturl1)
parsedjson1 = json.loads(loadedjson1.text)

In [0]:
geoidstring_part2 = ",".join(geoid[int(len(geoid)/2):])
requesturl2 = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (cidstring, geoidstring_part2)
loadedjson2 = requests.get(requesturl2)
parsedjson2 = json.loads(loadedjson2.text)

In [0]:
parsedjson = dict(parsedjson1['data'], **parsedjson2['data'])

In [0]:
parsedjson['31000US10180']['B03002']['estimate']

{'B03002001': 174006.0,
 'B03002002': 133580.0,
 'B03002003': 110633.0,
 'B03002004': 13466.0,
 'B03002005': 648.0,
 'B03002006': 3491.0,
 'B03002007': 0.0,
 'B03002008': 578.0,
 'B03002009': 4764.0,
 'B03002010': 706.0,
 'B03002011': 4058.0,
 'B03002012': 40426.0,
 'B03002013': 21001.0,
 'B03002014': 1075.0,
 'B03002015': 0.0,
 'B03002016': 461.0,
 'B03002017': 27.0,
 'B03002018': 16003.0,
 'B03002019': 1859.0,
 'B03002020': 871.0,
 'B03002021': 988.0}

In [0]:
HH_Income = []
Pop_by_age = []
Pop_by_Hispanic = []
for i, row in df.iterrows():
    Income = parsedjson[df['geoid'][i]]['B19001']['estimate']['B19001011']
    + parsedjson[df['geoid'][i]]['B19001']['estimate']['B19001012'] 
    HH_Income.append(Income)
    Pop = parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001007']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001008']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001009']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001010']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001011']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001012']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001031']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001032']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001033']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001034']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001035']
    + parsedjson[df['geoid'][i]]['B01001']['estimate']['B01001036']
    Pop_by_age.append(Pop)
    Hispanic = parsedjson[df['geoid'][i]]['B03002']['estimate']['B03002012']
    Pop_by_Hispanic.append(Hispanic)

df['Pop_by_Income'] = HH_Income
df['Pop_by_Age'] = Pop_by_age
df['Pop_by_Hispanic'] = Pop_by_Hispanic

In [0]:
#parameter 1
print(df.loc[df['Pop_by_Income'] == max(df['Pop_by_Income'])]['NAME'])

#parameter 2
print(df.loc[df['Pop_by_Age'] == max(df['Pop_by_Age'])]['NAME'])

#parameter 3
print(df.loc[df['Pop_by_Hispanic'] == max(df['Pop_by_Hispanic'])]['NAME'])


249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object
249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object
210    Los Angeles-Long Beach-Anaheim, CA
Name: NAME, dtype: object


In [0]:
HH_Income_Percentage = []
Pop_by_age_Percentage = []
Pop_by_Hispanic_Percentage = []
for i, row in df.iterrows():
    Percentage_Income = df['Pop_by_Income'][i]/max(df['Pop_by_Income'])
    Percentage_Age = df['Pop_by_Age'][i]/max(df['Pop_by_Age'])
    Percentage_Hispanic = df['Pop_by_Hispanic'][i]/max(df['Pop_by_Hispanic'])
    HH_Income_Percentage.append(Percentage_Income)
    Pop_by_age_Percentage.append(Percentage_Age)
    Pop_by_Hispanic_Percentage.append(Percentage_Hispanic)

df['HH_Income_Percentage'] = HH_Income_Percentage
df['Pop_by_age_Percentage'] = Pop_by_age_Percentage
df['Pop_by_Hispanic_Percentage'] = Pop_by_Hispanic_Percentage

mean_percentage_score = []
for i, row in df.iterrows():
    Mean = (df['HH_Income_Percentage'][i] + df['Pop_by_age_Percentage'][i]+ df['Pop_by_Hispanic_Percentage'][i])/3
    mean_percentage_score.append(Mean)
df['mean_percentage_score'] = mean_percentage_score




In [0]:
#highest mean percentage
print(df.loc[df['mean_percentage_score'] == max(df['mean_percentage_score'])]['NAME'])

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object
